![](images/GMIT_Logo.jpg)

# Higher Diploma in Science in Computing (Data Analytics)
### Programme Module: Machine Learning and Statistics (COMP08055) - Winter 21/22
---
## Scipy Stats Jupyter Notebook

" SciPy is an open-source scientific computing library for the Python programming language. Since its initial release in 2001, SciPy has become a de facto standard for leveraging scientific algorithms in Python, with over 600 unique code contributors, thousands of dependent packages, over 100,000 dependent repositories and millions of downloads per year".[10]

It is an open-source package that is a collection of "numerical algorithms and domain-specific toolboxes used in many mathematical, engineering and data research projects". SciPy is comparable with other scientific-computing libraries, including but not limited to GSL (GNU Scientific Library for C and C++) or MATLAB's toolboxes. Within Python, the SciPy module can undertake the vast majority of data analysis projects.[2] It can perform a multitude of statistical functions and tests, handling the majority of your analytical needs.

Of the many API's available within SciPy, there is a statistical API called "Stats", which contains a vast number of probability distributions and an increasing number of statistical functions, especially for probability function study. Within the SciPy Stats module, there is a range of other statistical function APIs to use, such as discrete distributions, summary statistics, correlation functions, transformations, Random Variate generation and plot tests, to mention but a few.[9]

## References

[1] Tutorialspoint.com. 2021. SciPy Tutorial. [online] Available at: <https://www.tutorialspoint.com/scipy/index.htm> [Accessed 21 October 2021].

[2] Yudha Wijaya, C., 2014. 3 Top Python Packages to Learn Statistic for Data Scientist. [online] Medium. Available at: <https://towardsdatascience.com/3-top-python-packages-to-learn-statistic-for-data-scientist-d753b76e6099> [Accessed 21 October 2021].

[3] Programming Review. 2021. Python Scipy Examples — Programming Review. [online] Available at: <https://programming-review.com/python/scipy-examples> [Accessed 21 October 2021].

[4] Biswas, D., 2021. Best Python Libraries For Data Science In 2021. [online] Analytics India Magazine. Available at: <https://analyticsindiamag.com/best-python-libraries-for-data-science-in-2021/> [Accessed 21 October 2021].

[5] L, J., 2021. SciPy Tutorial: How to Get Started With SciPy. [online] Medium. Available at: <https://liu-111.medium.com/scipy-tutorial-how-to-get-started-with-scipy-e4737000d77c> [Accessed 21 October 2021].

[6] Olson, R., 2021. Statistical analysis made easy in Python | Dr. Randal S. Olson. [online] Dr. Randal S. Olson | Musings of a data tinkerer. Available at: <http://www.randalolson.com/2012/08/06/statistical-analysis-made-easy-in-python/> [Accessed 21 October 2021].

[7] Scipy.org. 2021. Documentation — SciPy.org. [online] Available at: <https://www.scipy.org/docs.html> [Accessed 21 October 2021].

[8] Varoquaux, V., 2021. 1.6. Scipy : high-level scientific computing — Scipy lecture notes. [online] Scipy-lectures.org. Available at: <http://scipy-lectures.org/intro/scipy.html> [Accessed 21 October 2021].

[9] Nair, P., 2021. Simple Statistics with SciPy. [online] Comfort at 1 AU. Available at: <https://oneau.wordpress.com/2011/02/28/simple-statistics-with-scipy/> [Accessed 21 October 2021].

[10] Virtanen, P., 2020. SciPy 1.0: Fundamental Algorithms for Scientific Computing in Python. [online] Research Gate. Available at: <https://www.researchgate.net/publication/339008987_SciPy_10_fundamental_algorithms_for_scientific_computing_in_Python> [Accessed 21 October 2021].